<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/blackhole_entanglement_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cirq tensorflow_quantum

In [ ]:
#!/usr/bin/env python3
"""
blackhole_entanglement.py

AI-optimized analysis of two-qubit entanglement (toy black-hole model)
using TensorFlow Quantum (TFQ) and Cirq.
"""

import cirq
import sympy
import numpy as np
import tensorflow as tf
import tensorflow_quantum as tfq
from tensorflow.keras import layers, models, optimizers, losses

# 1) Build a Parameterized Entangling Circuit
def create_entanglement_circuit():
    # Two qubits on a grid
    q0, q1 = cirq.GridQubit(0, 0), cirq.GridQubit(0, 1)

    # Symbolic rotation parameters
    theta0, theta1 = sympy.symbols('theta0 theta1')

    # Circuit: Rx(θ0) on q0, Ry(θ1) on q1, then CNOT
    circuit = cirq.Circuit(
        cirq.rx(theta0)(q0),
        cirq.ry(theta1)(q1),
        cirq.CNOT(q0, q1)
    )

    # Readouts: measure Pauli Z on each qubit
    readouts = [cirq.Z(q0), cirq.Z(q1)]
    return circuit, readouts

# 2) Toy Dataset: random θ pairs → approximate concurrence label
def generate_dataset(n_samples=200):
    # Random angles in [0, 2π)
    angles = np.random.uniform(0, 2 * np.pi, size=(n_samples, 2))

    # Compute a toy “entanglement” label: concurrence ~ |sin(θ0 - θ1)|
    labels = np.abs(np.sin(angles[:, 0] - angles[:, 1]))
    labels = labels.reshape(-1, 1).astype(np.float32)

    # Build Cirq circuits with parameter assignment
    circuits = []
    for θ0, θ1 in angles:
        c, _ = create_entanglement_circuit()
        resolver = cirq.ParamResolver({'theta0': θ0, 'theta1': θ1})
        circuits.append(c.with_parameters(resolver))

    # Convert to TFQ tensor
    tq = tfq.convert_to_tensor(circuits)
    return tq, labels

# 3) Build Hybrid Quantum-Classical Model
def build_model(circuit, readouts):
    # PQC layer returns expectation values for each readout operator
    pqc = tfq.layers.PQC(circuit, readouts, differentiator=tfq.differentiators.Adjoint())

    # Chain classical layers on top
    model = models.Sequential([
        layers.Input(shape=(), dtype=tf.string),
        pqc,                            # → shape=(None, 2)
        layers.Dense(16, activation='relu'),
        layers.Dense(1)                 # regression output
    ])

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.01),
        loss=losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanAbsoluteError()]
    )
    return model

def main():
    # 1. Prepare circuit & readouts
    circuit, readouts = create_entanglement_circuit()

    # 2. Generate toy data
    X, y = generate_dataset(n_samples=500)
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # 3. Build & summarize model
    model = build_model(circuit, readouts)
    model.summary()

    # 4. Train
    model.fit(
        X_train, y_train,
        epochs=30,
        batch_size=32,
        validation_data=(X_test, y_test),
        verbose=2
    )

    # 5. Evaluate
    loss, mae = model.evaluate(X_test, y_test, verbose=0)
    print(f'\nTest Loss: {loss:.4f} │ Test MAE: {mae:.4f}')

    # 6. Inspect raw quantum expectations for a few samples
    pqc_layer = model.layers[1]
    sample_out = pqc_layer(X_test[:5])
    print("Quantum Expectations:\n", sample_out.numpy())

    # 7. Predict entanglement
    preds = model.predict(X_test[:5])
    print("Predicted Concurrence:", preds.squeeze())
    print("True Concurrence     :", y_test[:5].squeeze())

if __name__ == "__main__":
    main()